<a href="https://colab.research.google.com/github/jwilliams2023/Machine-Learning-Assignments/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [ ]:
%pip --quiet install numpy pandas matplotlib seaborn scikit-learn nltk gensim datasets transformers tensorflow keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==1

## Question 1 - Word Embeddings and N-gram

In [ ]:
import gensim.downloader as api
from gensim.models import FastText
from gensim.test.utils import common_texts

print(common_texts[0])
print(len(common_texts[0]))

ModuleNotFoundError: No module named 'numpy.strings'

### 1. (a)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd


In [ ]:
words = ["Dog", "Bark", "Tree", "Bank", "River", "Money"]

In [ ]:
def compute_similarity_matrix(model, words):
  vectors = []
  labels = []
  for word in words:
    key = word.lower()
    if key in model:
      vectors.append(model[key])
      labels.append(word)
    else:
      print(word, "not found in vocab")

  similarity_matrix = cosine_similarity(vectors)
  df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)
  return df

In [ ]:
if 'model' not in globals():
    model = api.load('glove-twitter-50')

In [ ]:
similarity_matrix = compute_similarity_matrix(model, words)
display(similarity_matrix)

### 1. (b)

In [ ]:
fasttext_model = FastText(sentences=common_texts, vector_size=50, window=5, min_count=1, epochs=10)
fasttext_similarity_matrix = compute_similarity_matrix(fasttext_model.wv,words)
display(fasttext_similarity_matrix)

### 1. (c)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.pyplot as plt

# Function to plot a heatmap from a similarity matrix
def plot_heatmap(sim_matrix, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(sim_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
    plt.title(title)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# Plot GloVe similarity heatmap
plot_heatmap(similarity_matrix, "GloVe Similarity Heatmap")

# Plot FastText similarity heatmap
plot_heatmap(fasttext_similarity_matrix, "FastText Similarity Heatmap")


#### Which embedding captures better semantics? Justify your answer

Based on the similarity matrices and visualizations, GloVe clearly captures better semantic relationships among the words compared to FastText in this case.

The GloVe model is pretrained on a large-scale Twitter corpus and contains well-established semantic patterns. For example:
- "Dog" and "Bark" have a similarity of 0.59, reflecting their real-world connection.
- "Bank" and "Money" score 0.67, indicating strong financial association.
- "Bank" and "River" also show moderate similarity at 0.32, acknowledging the alternate geographic meaning of "bank."

In contrast, the FastText model, although more flexible in handling out-of-vocabulary words and morphology through subword units, was trained here on a small built-in dataset (common_texts) that lacks real-world context. As a result:
- "Dog" and "Bark" only score 0.11
- "Bank" and "Money" show near-zero similarity (-0.02)
- Many relationships even have negative similarities, indicating noise rather than semantic closeness.

While FastText has the potential to outperform GloVe on domain-specific or rare words if trained on a richer corpus, in this case, GloVe significantly outperforms FastText in capturing meaningful semantic similarities.


### 2. N-grams and Classification (a)

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

nltk.download('twitter_samples')
nltk.download('stopwords')

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

all_tweets = positive_tweets + negative_tweets
labels = [1] * len(positive_tweets) + [0] * len(negative_tweets)

X_train, X_test, y_train, y_test = train_test_split(all_tweets, labels, test_size=0.3, random_state=77)


### 2. (b)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizers = {
    n: CountVectorizer(ngram_range=(n, n), stop_words='english')
    for n in range(1, 5)
}


### 2. (c)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

res = []

for n, vectorizer in vectorizers.items():
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    model = LogisticRegression(max_iter=200)
    model.fit(X_train_vec, y_train)

    y_pred = model.predict(X_test_vec)
    acc = accuracy_score(y_test, y_pred)

    res.append((n, acc))
    print(n, "-gram accuracy score", acc)

### 2.

In [ ]:
unzipped = list(zip(*res))

ngram_sizes, accuracy_scores = unzipped[0], unzipped[1]

plt.plot(ngram_sizes, accuracy_scores, marker="o")
plt.title("Accuracy vs n-gram size")
plt.xlabel("n-gram size")
plt.ylabel("Accuracy")
# plt.grid(True)
# plt.xticks(ngram_sizes)
plt.xticks(ngram_sizes, labels=["unigram = 1", "bigram = 2", "trigram = 3", "quadgram = 4"])
plt.show()

#### (d) How does the value of n in n-gram affect the model’s performance?

As the value of n increases, the model’s performance decreases. Unigrams (n = 1) gave the highest accuracy, while bigrams, trigrams, and quadgrams showed a steady drop in accuracy.

This is likely because larger n-grams lead to more sparse features. While bigrams and higher can capture more context, they also result in many rare combinations that the model sees too few times to learn from. This can cause overfitting and reduced generalization.

In this case, unigrams performed best due to their simplicity and broad coverage across the dataset.


## Question 2 RNN and Machine Translation

### 1.

In [ ]:
!pip install --quiet --upgrade transformers datasets evaluate rouge_score nltk

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")
billsum = billsum.train_test_split(test_size=0.2)
train = billsum["train"]
test = billsum["test"]

### 2.

### 3.

### 4.